In [13]:
#import libraries
#!pip install umap
#from umap import UMAP
import pandas as pd
import os 
import csv
import sklearn
import numpy as np
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.decomposition import KernelPCA

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score
from sklearn.cross_decomposition import CCA
from sklearn import neighbors
from skmultilearn.adapt import MLkNN
from sklearn import svm
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.feature_selection import SelectFromModel
from time import time
from sklearn.manifold import TSNE



In [2]:
#read metadata
meta=pd.read_csv('features/Metadata/dev_metadata.csv')
meta_test=pd.read_csv('features/Metadata/test_metadata.csv')
print(meta)
print(meta_test)

                    ID           artist                   song  \
0     ZEexJX2SztZYrOkD     Roger Waters                Déjà Vu   
1     nioxFqknx3fom1hP      Iron Maiden               Invaders   
2     B9jCiktNXTYQXrE7    The Offspring  Jennifer Lost the War   
3     XXz68KfaEjOJF7bw           Health           KNIGHTRIDERS   
4     QM8qykYFnIFYfJvS  Freeze Corleone             Jeremy Lin   
...                ...              ...                    ...   
3673  G6IjeJK8Hyyyz07u     Light Asylum          Shallow Tears   
3674  ZqiBxHOfpB9baZRA     mewithoutYou      East Enders Wives   
3675  V8hJK8mWXccbpxyX     Nina Nesbitt          Last December   
3676  m7gADmFpdvANqa6k             Ride           Kaleidoscope   
3677  jBInKb9WlU5RgwMJ              Sia     Midnight Decisions   

                                                  album  popularity  release  \
0                      Is This The Life We Really Want?        46.0     2017   
1             The Number of the Beast (2015 - R

In [3]:
#Datapreparation read all audio data 
from functools import reduce

audio_ivec=pd.read_csv('features/Audio/id_ivec512.csv')
audio_mfcc_mean=pd.read_csv('features/Audio/id_mfcc_mean_cov.csv')
audio_blf_chroma=pd.read_csv('features/Audio/id_chroma_bow.csv')
audio_blf_bow=pd.read_csv('features/Audio/id_mfcc_bow.csv')
audio_blf_emo_bow=pd.read_csv('features/Audio/id_emobase_bow.csv')


audio_blf_correlation=pd.read_csv('features/Audio/id_blf_correlation.csv')
audio_blf_logfluc=pd.read_csv('features/Audio/id_blf_logfluc.csv')
audio_blf_deltaspectral=pd.read_csv('features/Audio/id_blf_deltaspectral.csv')
audio_blf_spectralcontrast=pd.read_csv('features/Audio/id_blf_spectralcontrast.csv')
audio_blf_vardeltaspectral=pd.read_csv('features/Audio/id_blf_vardeltaspectral.csv')
audio_blf_spectral=pd.read_csv('features/Audio/id_blf_spectral.csv')


#PCA for all high dimensional features 

number_of_comp=500

columns = ['pca_comp_%i' % i
   for i in range(number_of_comp)
]

#pca deltaspectral

pca = PCA(n_components = number_of_comp).fit(audio_blf_deltaspectral.iloc[:,1:])
pca_=pd.DataFrame(pca.transform(audio_blf_deltaspectral.iloc[:,1:]), columns = columns, index = audio_blf_deltaspectral.iloc[:,1:].index)


audio_blf_deltaspectral_pca=pd.merge(audio_blf_deltaspectral.iloc[:,0], pca_, left_index=True, right_index=True)

#pca spectralcontrast

pca = PCA(n_components = number_of_comp).fit(audio_blf_spectralcontrast.iloc[:,1:])
pca_=pd.DataFrame(pca.transform(audio_blf_spectralcontrast.iloc[:,1:]), columns = columns, index = audio_blf_spectralcontrast.iloc[:,1:].index)


audio_blf_spectralcontrast_pca=pd.merge(audio_blf_spectralcontrast.iloc[:,0], pca_, left_index=True, right_index=True)


#pca spectral

pca = PCA(n_components = number_of_comp).fit(audio_blf_spectral.iloc[:,1:])
pca_=pd.DataFrame(pca.transform(audio_blf_spectral.iloc[:,1:]), columns = columns, index = audio_blf_spectral.iloc[:,1:].index)


audio_blf_spectral_pca=pd.merge(audio_blf_spectral.iloc[:,0], pca_, left_index=True, right_index=True)


#pca vardeltaspectral

pca = PCA(n_components = number_of_comp).fit(audio_blf_vardeltaspectral.iloc[:,1:])
pca_=pd.DataFrame(pca.transform(audio_blf_vardeltaspectral.iloc[:,1:]), columns = columns, index = audio_blf_vardeltaspectral.iloc[:,1:].index)


audio_blf_vardeltaspectral_pca=pd.merge(audio_blf_vardeltaspectral.iloc[:,0], pca_, left_index=True, right_index=True)

#pca logfluc

pca = PCA(n_components = number_of_comp).fit(audio_blf_logfluc.iloc[:,1:])
pca_=pd.DataFrame(pca.transform(audio_blf_logfluc.iloc[:,1:]), columns = columns, index = audio_blf_logfluc.iloc[:,1:].index)


audio_blf_logfluc_pca=pd.merge(audio_blf_logfluc.iloc[:,0], pca_, left_index=True, right_index=True)

#PCA correlation rythm 

#################

pca = PCA(n_components = number_of_comp).fit(audio_blf_correlation.iloc[:,1:])
pca_=pd.DataFrame(pca.transform(audio_blf_correlation.iloc[:,1:]), columns = columns, index = audio_blf_correlation.iloc[:,1:].index)

audio_blf_correlation_pca=pd.merge(audio_blf_correlation.iloc[:,0], pca_, left_index=True, right_index=True)



In [4]:
#prepare frame mfcc mean change rate

audio_mfcc_frames=pd.read_csv('features/Audio/id_frame_mfcc.csv')
audio_mfcc_frames_=audio_mfcc_frames
audio_mfcc_frames_.iloc[:,2:]=audio_mfcc_frames_.iloc[:,2:].diff()**2
hello=audio_mfcc_frames_.groupby(['ID'], as_index=False).sum()
hello=hello.drop(columns='frame')
print(hello.head())

#need to implement BOF

                 ID           0             1             2             3  \
0  00GCd9HYEge6Ntwi  982.456954  44228.668526  66809.511633  98501.125775   
1  01OcWE6ZDS0RAb4I  112.596728  12934.508389  30759.029167  50503.848806   
2  01UK70YxnzhjXY29   97.340392  16132.629774  37557.072857  51455.529299   
3  04elyRT3sHC2gYJK  121.809841  24879.904705  33416.528365  47053.649276   
4  04k3rCH5GDaZmhj0  184.464204  33197.220437  44028.347025  53628.118065   

               4              5              6              7              8  \
0  111127.250806  157984.695311  177905.792434  177317.806414  190182.745434   
1   77877.105539   91510.659925  117381.737711  136558.839864  167493.429929   
2   77917.108233   95041.055005  112850.760644  131092.701192  134267.005263   
3   76098.495296   92065.165509  120847.935946  139434.747254  136824.224230   
4   84538.617507   97556.916647  114746.585428  121355.210297  151081.940239   

               9             10             11          

In [48]:
#Data considered for regression

data_frames_1=[meta, audio_blf_bow, audio_blf_chroma, audio_ivec,
audio_mfcc_mean,
audio_blf_chroma,
audio_blf_emo_bow, video_vgg_19]

data_frames_1_test=[meta_test, audio_blf_bow, audio_blf_chroma, audio_ivec,
audio_mfcc_mean,
audio_blf_chroma,
audio_blf_emo_bow]
#data_frames_2=[audio_blf_correlation_pca, audio_blf_deltaspectral_pca, audio_blf_logfluc_pca] #audio_blf_spectralcontrast_pca, audio_blf_vardeltaspectral_pca, audio_blf_spectral_pca]


data_frames_2=[audio_blf_logfluc, audio_blf_correlation, audio_blf_deltaspectral,
               audio_blf_spectralcontrast, audio_blf_vardeltaspectral, audio_blf_spectral]

data_frames=data_frames_1+data_frames_2

data_frames_test=data_frames_1_test+data_frames_2

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['ID'],
                                            how='inner'), data_frames)


df_merged_test = reduce(lambda  left,right: pd.merge(left,right,on=['ID'],
                                            how='inner'), data_frames_test)



#ca = CCA()
#ca.fit(audio_mfcc_mean.iloc[:, 1:], vgg_19.iloc[:, 1:])
#X_c, Y_c = ca.transform(audio_mfcc_mean.iloc[:, 1:], vgg_19.iloc[:, 1:])


In [53]:
#Regression task, approach feature selection with linear regression and linear regression prediction

#Train data
X=df_merged.iloc[:,5:]

#Normalize data 
scaler = StandardScaler()
X= scaler.fit_transform(X)


pca_k=KernelPCA(n_components=500, kernel='rbf')
projections=pca_k.fit_transform(X)

#downproject data with PCA, not better
#number_of_comp=50
#columns = ['pca_comp_%i' % i
 #  for i in range(number_of_comp)]
#pca = PCA(n_components = number_of_comp).fit(df_merged.iloc[:,6:])
#X=pd.DataFrame(pca.transform(df_merged.iloc[:,6:]), columns = columns, index = df_merged.iloc[:,6:].index)

#Labels

y=df_merged['popularity']

#feature selection does not work
#univariate
#X_train, X_test, y_train, y_test=train_test_split(X, y, random_state=0, test_size=0.5)
#select=SelectPercentile(percentile=1)
#select.fit(X, y)
#X_selected=select.transform(X)

#model-based feature selection
select=SelectFromModel(LinearRegression(), threshold='4*mean')#5 mean
select.fit(X, y)
X_selected_2=select.transform(X)

# print shape of train data
print(X_selected_2.shape)
print(X.shape)
#Cross-validation
#clf=LinearRegression()

C_=[0.001, 0.1,1,10, 100]
gamma_=[0.001, 0.1,1, 10, 100]
for C in C_:
    for gamma in gamma_:
        clf=SVR(C=C, gamma=gamma)
        #clf=RandomForestRegressor(n_estimators=1001)
        scores=cross_validate(clf, X_selected_2, y, scoring=['neg_mean_absolute_error',
        'neg_root_mean_squared_error'])
        print(C, gamma, scores)

clf=LinearRegression()

scores=cross_validate(clf, projections, y, scoring=['neg_mean_absolute_error',
        'neg_root_mean_squared_error'])
scores


#clf.fit(X_selected_2, y)


        

(3678, 117)
(3678, 19952)
0.001 0.001 {'fit_time': array([2.0127275 , 1.77396798, 1.81513047, 1.97965097, 1.81899428]), 'score_time': array([0.85602593, 0.89795637, 0.89508677, 0.96634865, 0.91629028]), 'test_neg_mean_absolute_error': array([-11.23192399, -11.71306767, -11.45961275, -10.90541655,
       -11.41127412]), 'test_neg_root_mean_squared_error': array([-14.12767006, -14.40593715, -14.29964815, -13.89822673,
       -14.5831173 ])}
0.001 0.1 {'fit_time': array([1.90983891, 1.82010674, 1.78284764, 1.8716352 , 2.07935882]), 'score_time': array([1.00395203, 0.88741994, 0.8872788 , 0.82267046, 1.06158328]), 'test_neg_mean_absolute_error': array([-11.23372232, -11.7149667 , -11.46144336, -10.90819968,
       -11.4132494 ]), 'test_neg_root_mean_squared_error': array([-14.13008039, -14.40833688, -14.30180095, -13.90183552,
       -14.58594697])}
0.001 1 {'fit_time': array([4.16679811, 2.36106133, 2.19318938, 2.69990277, 2.44431567]), 'score_time': array([1.25188041, 1.13363481, 1.07212

100 0.001 {'fit_time': array([1.32249165, 1.3686924 , 1.19959331, 1.15616941, 1.15879869]), 'score_time': array([0.77643847, 0.43621707, 0.47780561, 0.45525432, 0.45313096]), 'test_neg_mean_absolute_error': array([-10.63763096, -10.92709754, -10.56594592, -10.24380858,
       -10.71885018]), 'test_neg_root_mean_squared_error': array([-13.29389641, -13.51107789, -13.28413006, -12.86011419,
       -13.45203319])}
100 0.1 {'fit_time': array([1.35096812, 1.52974725, 1.22730947, 1.42690754, 1.62649226]), 'score_time': array([0.56374097, 0.44626141, 0.41928864, 0.43475199, 0.62485528]), 'test_neg_mean_absolute_error': array([-11.27228883, -11.73996216, -11.5031171 , -10.88427409,
       -11.36986584]), 'test_neg_root_mean_squared_error': array([-14.12889757, -14.3861861 , -14.28475653, -13.85656839,
       -14.49690696])}
100 1 {'fit_time': array([1.601753  , 1.57834816, 1.35197854, 1.52297401, 1.51299953]), 'score_time': array([0.51800346, 0.42231679, 0.43222785, 0.44287682, 0.45239091]), '

{'fit_time': array([6.06827784, 1.84154606, 2.95398092, 0.69906616, 0.60319567]),
 'score_time': array([0.00399661, 0.01198626, 0.0039947 , 0.00399399, 0.00399494]),
 'test_neg_mean_absolute_error': array([-10.98729759, -11.52728301, -11.02354709, -11.42946869,
        -10.96796896]),
 'test_neg_root_mean_squared_error': array([-13.71745797, -14.50222408, -13.78671476, -14.30151305,
        -13.69302116])}

In [15]:
#Baseline for regression

print(scores)

baseline=np.full(y.shape, np.mean(y))
print('Mean absolute error {0}:'.format(mean_absolute_error(y, baseline)))
print('Mean squared error {0}:'.format(mean_squared_error(y, baseline, squared=False)))


{'fit_time': array([0.04188848, 0.05086565, 0.0398922 , 0.036901  , 0.04188657]), 'score_time': array([0.00897622, 0.00199366, 0.00199461, 0.00199556, 0.00199556]), 'test_neg_mean_absolute_error': array([-10.91211551, -11.1162842 , -11.11951655, -10.54948933,
       -10.84785627]), 'test_neg_root_mean_squared_error': array([-13.75800114, -13.77126853, -13.91362917, -13.26259626,
       -13.74425904])}
Mean absolute error 11.358564111935198:
Mean squared error 14.239037505874942:


In [ ]:
##predictions

X_pred=df_merged_test.iloc[:,4:]
scaler = StandardScaler()
X_pred= scaler.fit_transform(X_pred)

X_pred=select.transform(X_pred)

print(X.shape)
print(X_pred.shape)
#Normalize data 


prediction=pd.DataFrame(clf.predict(X_pred))
predictions_df=pd.concat([meta_test['ID'], prediction], axis=1)

print(predictions_df)

In [45]:
#Data frames for tagging. Handgrafting shows that this combination gives the best result (Parameter tune?)


data_frames_1=[meta, audio_blf_chroma, audio_blf_bow, audio_ivec, audio_blf_emo_bow, video_vgg_19]
#data_frames_2=[audio_blf_logfluc_pca, audio_blf_correlation_pca,
 #               audio_blf_spectralcontrast_pca]

data_frames_2=[audio_blf_logfluc, audio_blf_correlation, audio_blf_deltaspectral,
              audio_blf_spectralcontrast, audio_blf_vardeltaspectral, audio_blf_spectral]

data_frames=data_frames_1+data_frames_2
#audio_blf_bow good, logfluc does something, audio_blf (PCA 500; ca.10)
#
#print(vgg_19)

#
#data_frames=[meta, audio_mfcc_mean, audio_blf_bow] 
            # video_vgg_19]

data_frames_test=[meta_test, audio_mfcc_mean, audio_blf_bow]

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['ID'],
                                            how='inner'), data_frames)


df_merged_test = reduce(lambda  left,right: pd.merge(left,right,on=['ID'],
                                            how='inner'), data_frames_test)


In [46]:
#Tagging

#preparing tag data, PCA down-projection did not work 

#number_of_comp=500
#columns = ['pca_comp_%i' % i
#for i in range(number_of_comp)]
#pca = PCA(n_components = number_of_comp).fit(df_merged.iloc[:,6:])
#X=pd.DataFrame(pca.transform(df_merged.iloc[:,6:]), columns = columns, index = df_merged.iloc[:,6:].index)

#define and prepare tags for multilabel classification 

tags_df=pd.read_csv('features/Metadata/dev_tags.csv')

tags_df['tags_new']=tags_df['tags']


for ii in range(len(tags_df['tags'])):
    tagi=tags_df['tags'][ii]
    tagi=tagi.replace('[','')
    tagi=tagi.replace(']','')
    tagi=tagi.replace(' ','')
    tagi=tagi.split(",")
    tags_df['tags_new'][ii]=tagi
    

df_merged_tags=df_merged.merge(tags_df, on=['ID'])

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_merged_tags['tags_new'])


# normalize features and transform target variable

X=df_merged_tags.iloc[:,5:-2]
scaler = StandardScaler()
scaler.fit(X)
X= scaler.transform(X)

pca_k=KernelPCA(n_components=100, kernel='rbf')
projections=pca_k.fit_transform(X)

y = multilabel_binarizer.transform(df_merged_tags['tags_new'])

#feature selection did not work 
#from sklearn.feature_selection import chi2, SelectKBest

#selected_features = [] 
#for label in range(y.shape[1]):
 #   selector = SelectKBest(chi2, k='all')
  #  selector.fit(X, y[:,label])
   # selected_features.append(list(selector.scores_))

#print(selected_features)

#selected_features = np.mean(selected_features, axis=0) > 0.3
#selected_features = np.max(selected_features, axis=0) > threshold

#make predictions

#lr=svm.SVC()
#clf = MultiOutputClassifier(svm.SVC()).fit(X, y)
lr = LogisticRegression(max_iter=100)
#lr = neighbors.KNeighborsClassifier(15)
clf_lab = OneVsRestClassifier(lr)

scores=cross_validate(clf_lab, projections, y, scoring=['f1_micro', 'precision_micro',
'recall_micro'])

scores

#clf_lab.fit(X,y)

#clf = neighbors.KNeighborsClassifier(15)
#clf = MLkNN(k=15)


#Define baseline 





c:\users\p41999\appdata\local\programs\python\python39\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 183 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
c:\users\p41999\appdata\local\programs\python\python39\lib\site-packages\sklearn\multiclass.py:80: UserWarning: Label not 72 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %


{'fit_time': array([7.60992646, 6.73838615, 6.22823429, 6.36478734, 6.34550261]),
 'score_time': array([0.23517537, 0.16807818, 0.15234303, 0.1499486 , 0.15691924]),
 'test_f1_micro': array([0.02521566, 0.02713178, 0.02967742, 0.02238315, 0.03006536]),
 'test_precision_micro': array([0.67857143, 0.72413793, 0.6969697 , 0.68      , 0.79310345]),
 'test_recall_micro': array([0.01284652, 0.01382488, 0.0151615 , 0.01137885, 0.01532312])}

In [26]:
#predicting labels 
print(projections.shape)

X_pred=df_merged_test.iloc[:,4:]
scaler = StandardScaler()
scaler.fit(X_pred)
X_pred= scaler.transform(X_pred)

prediction=clf_lab.predict(X_pred)
#prediction=multilabel_binarizer.inverse_transform(prediction)

predictions_df=meta_test['ID']

prediction=pd.DataFrame(prediction)
predictions_df=pd.concat([meta_test['ID'], prediction], axis=1)
print(predictions_df)


(3678, 100)


NotFittedError: This OneVsRestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [47]:
#Tunning decision threshold

for t in [0.01, 0.05, 0.1, 0.15, 0.17, 0.20, 0.25, 0.30, 0.40, 0.5]: 
    # threshold value

    X_train, X_test, y_train, y_test=train_test_split(projections, y, random_state=0, test_size=0.1)
    clf_lab.fit(X_train, y_train)
    y_pred_prob = clf_lab.predict_proba(X_test)
    y_pred_new = (y_pred_prob >= t).astype(int)
    print(t, f1_score(y_test, y_pred_new, average="micro"))
    #scores


print(y.mean(axis=1).mean())

0.01 0.07721186334626454
0.05 0.21631530705774518
0.1 0.25998874507597075
0.15 0.2534351145038167
0.17 0.25102543068088595
0.2 0.22121486854034453
0.25 0.17961654894046417
0.3 0.1471861471861472
0.4 0.06954436450839328
0.5 0.027568922305764413
0.010209352909189777


In [ ]:
#multilabel_binarizer.inverse_transform(y_pred)

#Baseline label

print(y.mean(axis=0))
print(y.sum(axis=1))
print(y.shape)
#for ii in 
dummy=np.zeros(y.shape)
for ii in range(y.shape[1]):
    proba=y.mean(axis=0)[ii]
    dummy[:,ii]=np.random.binomial(1, proba, y.shape[0])

print('Baseline f1: {0}'.format(f1_score(y, dummy, average='micro')))
print('Baseline precision: {0}'.format(precision_score(y, dummy, average='micro')))
print('Baseline recall: {0}'.format(recall_score(y, dummy, average='micro')))


dummy_clf = DummyClassifier(strategy="most_frequent")
clf_d = OneVsRestClassifier(dummy_clf)
clf_d.fit(X, y)
clf_d.predict(X)

print(clf_d.score(X, y))



In [ ]:
#issue predictions on the test set

#Merge with test IDs
#Adjust threshold in crossvalidation
#BOF


In [ ]:
from math import sqrt
from random import randrange
from random import seed
 
# calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += (row1[i] - row2[i])**2
    return sqrt(distance)
 
# Locate the best matching unit
def get_best_matching_unit(codebooks, test_row):
    distances = list()
    for codebook in codebooks:
        dist = euclidean_distance(codebook, test_row)
        distances.append((codebook, dist))
    distances.sort(key=lambda tup: tup[1])
    return distances[0][0]
 
# Create a random codebook vector
def random_codebook(train):
    n_records = len(train)
    n_features = len(train[0])
    codebook = [train[randrange(n_records)][i] for i in range(n_features)]
    return codebook
 
# Train a set of codebook vectors
def train_codebooks(train, n_codebooks, lrate, epochs):
    codebooks = [random_codebook(train) for i in range(n_codebooks)]
    for epoch in range(epochs):
        rate = lrate * (1.0-(epoch/float(epochs)))
        sum_error = 0.0
        for row in train:
            bmu = get_best_matching_unit(codebooks, row)
            for i in range(len(row)-1):
                error = row[i] - bmu[i]
                sum_error += error**2
                if bmu[-1] == row[-1]:
                    bmu[i] += rate * error
                else:
                    bmu[i] -= rate * error
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, rate, sum_error))
    return codebooks
 
# Test the training function
seed(1)
dataset = [[2.7810836,2.550537003,0],
           [1.465489372,2.362125076,0],
    [3.396561688,4.400293529,0],
    [1.38807019,1.850220317,0],
    [3.06407232,3.005305973,0],
    [7.627531214,2.759262235,1],
    [5.332441248,2.088626775,1],
    [6.922596716,1.77106367,1],
    [8.675418651,-0.242068655,1],
    [7.673756466,3.508563011,1]]
learn_rate = 0.3
n_epochs = 10
n_codebooks = 500



codebooks = train_codebooks(audio_mfcc_frames.iloc[:, 1:].values.tolist(), n_codebooks, learn_rate, n_epochs)
print('Codebooks: %s' % codebooks)

In [ ]:
#blf correlation
audio_blf_correlation=pd.read_csv('features/Audio/id_blf_correlation.csv')

print(audio_blf_correlation)
audio_blf_correlation['mean_cor']=audio_blf_correlation.mean(axis=1)
audio_blf_correlation['std_cor']=audio_blf_correlation.std(axis=1)

meta_audio=meta.merge(audio_blf_correlation, on='ID')

print(meta_audio)
X=meta_audio.iloc[:,14:]

y=meta_audio['popularity']

In [42]:
#Video 

video_vgg_19=pd.read_csv('features/id_vgg19_agg.csv')
meta_video=video_vgg_19.merge(meta, on='ID')
meta_video
X=meta_video.iloc[:,1:3000]
y=meta_video['popularity']

reg = LinearRegression().fit(X, y)
reg.score(X, y)

#low effect size


0.844682884200431

In [ ]:
#Prepare textdata
data_dir='features/Text/Lyrics'

paths=os.listdir(data_dir)
data = []

#for ii in range(3): 
for ii in range(len(paths)): 
    full_path = os.path.join(data_dir, paths[ii])
    with open(full_path, 'r', encoding="utf8") as file:
        dat = file.read().replace('\n', '')
    data.append([paths[ii].split('.')[0], dat])

df = pd.DataFrame(data, columns=['ID', 'Lyric'])

df.merge(meta, on='ID')
#Merging of dataframes 


vgg_19=pd.read_csv('features/id_vgg19_agg.csv')

number_of_comp=500

columns = ['pca_comp_%i' % i
   for i in range(number_of_comp)
]

#pca vgg_19

pca = PCA(n_components = number_of_comp).fit(vgg_19.iloc[:,1:])
pca_=pd.DataFrame(pca.transform(vgg_19.iloc[:,1:]), columns = columns, index = vgg_19.iloc[:,1:].index)
vgg_19_pca=pd.merge(vgg_19.iloc[:,0], pca_, left_index=True, right_index=True)
